In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 7.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

##Importing Required Libraries

In [2]:
import pandas as pd
import numpy as np
import pickle
import re
from datasets import load_dataset

from datasets import Dataset
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter
from sklearn.model_selection import train_test_split
from transformers import pipeline

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

from collections import defaultdict
from collections import Counter


from sklearn.metrics import (
    precision_score, 
    recall_score, 
    f1_score, 
    classification_report,
    accuracy_score
)
import evaluate
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback

In [3]:
df = pd.read_csv("bbc-text.csv")
labels = ['business', 'entertainment', 'politics', 'sport', 'tech']
df = df.dropna(how="any", axis=1)
df.columns = ['labels', 'text']
df['labels'] = df['labels'].apply(lambda x: labels.index(x))
df.head()

,labels,text
0,4,tv future in the hands of viewers with home th...
1,0,worldcom boss left books alone former worldc...
2,3,tigers wary of farrell gamble leicester say ...
3,3,yeading face newcastle in fa cup premiership s...
4,1,ocean s twelve raids box office ocean s twelve...


## Converting our Custom Dataset to HuggingFace Dataset

In [15]:
y = df["labels"]
X=df["text"]

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3,stratify=y)

In [16]:
df_train = pd.DataFrame(data=np.column_stack((y_train,X_train)),columns=["labels","text"])
df_test = pd.DataFrame(data=np.column_stack((y_test,X_test)),columns=["labels","text"])

print(df_train.shape)
print(df_test.shape)

(1557, 2)
(668, 2)


In [17]:
df_train.to_pickle("./df_train.pkl")  
df_test.to_pickle("./df_test.pkl") 

In [18]:
from datasets import Dataset, DatasetDict

bbc_news_dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "test": Dataset.from_pandas(df_test)
    })

In [19]:
bbc_news_dataset['train'][0]

{'labels': 0,
 'text': 'yukos seeks court action on sale yukos will return to a us court on wednesday to seek sanctions against baikal finance group  the little-known firm which has bought its main asset.  yukos has said it will sue baikal and others involved in the sale of yuganskneftegas for $20bn in damages. yukos  us lawyers will attempt to have baikal assets frozen after the russian government ignored a us court order last week blocking the sale. baikal s background and its motives for buying the unit are still unclear.  russian newspapers have claimed that baikal - which bought the yuganskneftegas production unit for $9.4bn (261bn roubles  £4.8bn) on sunday at a state provoked auction - has strong links with surgutneftegas  russia s fourth-biggest oil producer. many observers believe that the unit  which produces 60% of yukos  oil output  could ultimately fall into the hands of surgutneftegas or even gazprom  the state gas firm which opted out of the auction.  the russian governm

In [10]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [20]:
# loading tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [21]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [22]:
# tokenizing our dataset
tokenized_dataset = bbc_news_dataset.map(preprocess_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [23]:
tokenized_dataset['train']

Dataset({
    features: ['labels', 'text', 'input_ids', 'attention_mask'],
    num_rows: 1557
})

Now creating a batch of examples using DataCollatorWithPadding since it is more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximium length.

In [24]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [25]:
accuracy = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall = evaluate.load('recall')
precision = evaluate.load("precision")

In [26]:
#from datasets import load_metric

In [27]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": accuracy.compute(predictions=predictions, references=labels,)["accuracy"],
            "f1-score" : f1_metric.compute(predictions=predictions, references=labels,average='macro')["f1"],
            "recall" : recall.compute(predictions=predictions, references=labels,average='macro')["recall"],
            "precision" : precision.compute(predictions=predictions, references=labels,average='macro')["precision"]
    }




In [28]:
id2label = {0: "business", 1: "entertainment",2:"politics",3:"sport",4:"tech"}
label2id = {'business':0, 'entertainment':1, 'politics':2, 'sport':3, 'tech':4}


In [29]:
# Loading model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=5, id2label=id2label, label2id=label2id
)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifi

In [30]:
training_args = TrainingArguments(
    output_dir="distilbert-bbc-news-classification",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


Cloning https://huggingface.co/Umesh/distilbert-bbc-news-classification into local empty directory.


Download file pytorch_model.bin:   0%|          | 24.0k/255M [00:00<?, ?B/s]

Download file runs/Jan22_11-43-01_91fe605d2d43/events.out.tfevents.1674387798.91fe605d2d43.526.0: 100%|#######…

Download file runs/Jan22_11-43-01_91fe605d2d43/1674387798.4370775/events.out.tfevents.1674387798.91fe605d2d43.…

Download file runs/Jan22_12-18-56_91fe605d2d43/events.out.tfevents.1674389942.91fe605d2d43.526.4: 100%|#######…

...

Download file runs/Jan22_12-08-30_91fe605d2d43/events.out.tfevents.1674389320.91fe605d2d43.526.2: 100%|#######…

Clean file runs/Jan22_12-08-30_91fe605d2d43/events.out.tfevents.1674389320.91fe605d2d43.526.2:  26%|##6       …

Download file training_args.bin: 100%|##########| 3.37k/3.37k [00:00<?, ?B/s]

Clean file training_args.bin:  30%|##9       | 1.00k/3.37k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/255M [00:00<?, ?B/s]

In [31]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1557
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 294
  Number of trainable parameters = 66957317
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster th

Epoch,Training Loss,Validation Loss,Accuracy,F1-score,Recall,Precision
1,No log,0.149478,0.977545,0.977531,0.977207,0.978063
2,No log,0.073723,0.988024,0.988224,0.988531,0.987985
3,No log,0.066922,0.988024,0.988015,0.988597,0.987540


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 668
  Batch size = 16
Saving model checkpoint to distilbert-bbc-news-classification/checkpoint-98
Configuration saved in distilbert-bbc-news-classification/checkpoint-98/config.json
Model weights saved in distilbert-bbc-news-classification/checkpoint-98/pytorch_model.bin
tokenizer config file saved in distilbert-bbc-news-classification/checkpoint-98/tokenizer_config.json
Special tokens file saved in distilbert-bbc-news-classification/checkpoint-98/special_tokens_map.json
tokenizer config file saved in distilbert-bbc-news-classification/tokenizer_config.json
Special tokens file saved in distilbert-bbc-news-classification/special_tokens_map.json
The following columns in the e

TrainOutput(global_step=294, training_loss=0.28354844592866446, metrics={'train_runtime': 346.7628, 'train_samples_per_second': 13.47, 'train_steps_per_second': 0.848, 'total_flos': 618644722712310.0, 'train_loss': 0.28354844592866446, 'epoch': 3.0})

In [32]:
trainer.push_to_hub()

Saving model checkpoint to distilbert-bbc-news-classification
Configuration saved in distilbert-bbc-news-classification/config.json
Model weights saved in distilbert-bbc-news-classification/pytorch_model.bin
tokenizer config file saved in distilbert-bbc-news-classification/tokenizer_config.json
Special tokens file saved in distilbert-bbc-news-classification/special_tokens_map.json


Upload file runs/Jan23_22-23-14_00ac088df0f5/events.out.tfevents.1674512694.00ac088df0f5.679.0: 100%|#########…

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Umesh/distilbert-bbc-news-classification
   2a69519..bde3fb2  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Umesh/distilbert-bbc-news-classification
   2a69519..bde3fb2  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9880239520958084}, {'name': 'Recall', 'type': 'recall', 'value': 0.9885973856209151}, {'name': 'Precision', 'type': 'precision', 'value': 0.9875399610438732}]}
To https://huggingface.co/Umesh/distilbert-bbc-news-classification
   bde3fb2..f3c215c  main -> main

   bde3fb2..f3c215c  main -> main



'https://huggingface.co/Umesh/distilbert-bbc-news-classification/commit/bde3fb2cbcaa4cfe08948ac6ef73a03cc5d5795e'

In [33]:
text="The foldable mobile phone market is now gaining more attention but there are only a few players in the market. A few years ago, we had Huawei and Samsung as the major players in the Android field. However, after the ban on Huawei, Samsung became the sole player technically. Although other brands like Moto, Xiaomi and others have foldable phones, they are not as consistent as Samsung. As of now, there is no iOS foldable device and any may not launch in the near future. This gives Samsung more time to get ahead in the market. A new report reveals that Samsung is working on a new display that can rotate 360 degrees."

In [34]:


classifier = pipeline("text-classification", model="Umesh/distilbert-bbc-news-classification")
classifier(text)


Downloading:   0%|          | 0.00/851 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Umesh--distilbert-bbc-news-classification/snapshots/f3c215cecbe7f364c0f6ae0a745fc3cb1b695198/config.json
Model config DistilBertConfig {
  "_name_or_path": "Umesh/distilbert-bbc-news-classification",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "business",
    "1": "entertainment",
    "2": "politics",
    "3": "sport",
    "4": "tech"
  },
  "initializer_range": 0.02,
  "label2id": {
    "business": 0,
    "entertainment": 1,
    "politics": 2,
    "sport": 3,
    "tech": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": t

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--Umesh--distilbert-bbc-news-classification/snapshots/f3c215cecbe7f364c0f6ae0a745fc3cb1b695198/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at Umesh/distilbert-bbc-news-classification.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


Downloading:   0%|          | 0.00/360 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/711k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--Umesh--distilbert-bbc-news-classification/snapshots/f3c215cecbe7f364c0f6ae0a745fc3cb1b695198/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--Umesh--distilbert-bbc-news-classification/snapshots/f3c215cecbe7f364c0f6ae0a745fc3cb1b695198/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--Umesh--distilbert-bbc-news-classification/snapshots/f3c215cecbe7f364c0f6ae0a745fc3cb1b695198/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--Umesh--distilbert-bbc-news-classification/snapshots/f3c215cecbe7f364c0f6ae0a745fc3cb1b695198/tokenizer_config.json


[{'label': 'tech', 'score': 0.955810546875}]